In [3]:
import pandas as pd #might want to use it
import numpy as np #working with arrays/matrices after all! 
import gensim # Word embeddings
import random #random choice for bot's first word
from sklearn.metrics.pairwise import cosine_similarity
import convergence_resources as cr

Thoughts: How many times does a word appear? The computer should only choose the most common words. It can't repeat any of the user_input or the bot_input.

Miles: look up most common english words in a word2vec model, use the top 20,000 words? 10,000? https://wordcounter.io/blog/how-many-words-does-the-average-person-know/   

Distance: This could be helpful: https://stackoverflow.com/questions/32446703/find-closest-vector-from-a-list-of-vectors-python

# Using GloVe

In [4]:
#using numpy 
import numpy as np
def load_glove_model(file):
    print("Loading...")
    f = open(file,'r')
    model = {}
    for line in f:
        whole_line = line.split()
        words = whole_line[0]
        embedding = np.array([float(val) for val in whole_line[1:]])
        model[words] = embedding
    print("Done.",len(model)," words loaded!")
    return model
#help from https://stackoverflow.com/questions/37793118/load-pretrained-glove-vectors-in-python

In [5]:
model = load_glove_model('glove.6B/glove.6B.300d.txt')

Loading...
Done. 400000  words loaded!


# Common English Words

Create a list of common english words from https://www.ef.edu/english-resources/english-vocabulary/top-3000-words/ with help from https://www.reddit.com/r/learnpython/comments/1zcbi4/is_there_a_way_to_get_a_list_of_common_english/

In [6]:
with open("common_words.txt") as f:
    cwordlist = []
    for line in f:
        cwordlist.append(line.strip().lower().replace('-',''))

In [7]:
common_word_vectors = {}
unvectorized_common_words = []
for word in cwordlist:
    if word in model.keys():
        common_word_vectors[word] = model[word]
    else:
        unvectorized_common_words.append(word)

In [8]:
#reshape the dictionary arrays 
common_word_vectors['truth'].shape

for key, value in common_word_vectors.items():
    common_word_vectors[key] = value.reshape(1,-1)
    
for key, value in model.items():
    model[key] = value.reshape(1,-1)

common_word_vectors['truth'].shape

(1, 300)

# Create Functions

In [24]:
def converge(user_input=str, bot_input=str, exclude=None):
    """Return the "average word" of the input words."""
    if exclude is None:
        exclude = set()
    exclude.add(user_input)
    exclude.add(bot_input)  
    mean_vector = ((model[user_input] + model[bot_input])/2)
    cos_sim_dict = {}
    response_options_dict = {key: common_word_vectors[key] for key in common_word_vectors 
                             if key not in exclude}
    for word, vector in response_options_dict.items():
        cos_sim_dict[float(cosine_similarity(mean_vector, vector))] = word
    max_cos_sim = max(cos_sim_dict.keys())
    bot_response = cos_sim_dict[max_cos_sim]
    return bot_response

In [25]:
def play_round(user_input, user_history=None, bot_history=None):
    if bot_history is None:
        user_history = []
        bot_history = []
        bot_response = random.choice(cwordlist)
    else:
        bot_response = converge(user_history[-1], bot_history[-1], 
                                exclude=set(user_history + bot_history))
    user_history.append(user_input)
    bot_history.append(bot_response)
    return {
        'user_history': user_history,
        'bot_history': bot_history,
        'bot_response': bot_response,
    }

In [26]:
converge('cat', 'dog')

'pet'

In [40]:
play_round('ring', user_history=['bus', 'government', 'compromise', 'discuss', 'engagement'], 
           bot_history=['agreement', 'deal', 'plan', 'proposal', 'proposed'] )

{'user_history': ['bus',
  'government',
  'compromise',
  'discuss',
  'engagement',
  'ring'],
 'bot_history': ['agreement',
  'deal',
  'plan',
  'proposal',
  'proposed',
  'propose'],
 'bot_response': 'propose'}